In [154]:
import numpy as np
import pandas as pd

from arch import arch_model
from arch.univariate import EGARCH
import matplotlib.pyplot as plt
import statsmodels.api as sm

# local module
from data_cleaning.import_data import import_data
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [182]:
pairs = ['USDRUB', 'USDZAR', 'USDBRL', 'EURUSD', 'USDJPY', 'GBPUSD']
model = {}
aic = {}
bic = {}
loglik = {}
lbox = {}
noarch = {}

for pair in pairs:
     df = import_data(fx_pair=pair)
     df.dropna(inplace=True)
     returns = df['log_ret'] * 100
     returns[0] = 0.000
     garch = arch_model(returns, mean='Zero', vol='GARCH', p=1,q=1, dist = 'Studentst')
     garch_fit = garch.fit()
     lbox_test = sm.stats.acorr_ljungbox(garch_fit.resid, lags=[1], return_df=False)
     
     model[pair] = 'GARCH(1,1)'
     aic[pair] = np.round(garch_fit.aic,3)
     bic[pair] = np.round(garch_fit.bic,3)
     loglik[pair] = np.round(garch_fit.loglikelihood,3)
     lbox[pair] = np.round(lbox_test[1][0],3)

garchdf = pd.DataFrame([model,aic,bic,loglik,lbox], index=['Model','AIC', 'BIC', 'LogL','Ljung-Box (1)'])

Iteration:      1,   Func. Count:      6,   Neg. LLF: 14789.607038698148
Iteration:      2,   Func. Count:     12,   Neg. LLF: 8362.843192242908
Iteration:      3,   Func. Count:     18,   Neg. LLF: 8895.740237900996
Iteration:      4,   Func. Count:     24,   Neg. LLF: 3898.860921061597
Iteration:      5,   Func. Count:     29,   Neg. LLF: 4453.484440430397
Iteration:      6,   Func. Count:     36,   Neg. LLF: 9750.95831115683
Iteration:      7,   Func. Count:     45,   Neg. LLF: 5620.378613928
Iteration:      8,   Func. Count:     51,   Neg. LLF: 3864.030221736276
Iteration:      9,   Func. Count:     56,   Neg. LLF: 3858.0814855274016
Iteration:     10,   Func. Count:     61,   Neg. LLF: 3856.6672465706433
Iteration:     11,   Func. Count:     66,   Neg. LLF: 3855.9349403845417
Iteration:     12,   Func. Count:     71,   Neg. LLF: 3855.7970681427446
Iteration:     13,   Func. Count:     76,   Neg. LLF: 3855.78037805301
Iteration:     14,   Func. Count:     81,   Neg. LLF: 3855.77804

In [183]:
pairs = ['USDRUB', 'USDZAR', 'USDBRL', 'EURUSD', 'USDJPY', 'GBPUSD']
model = {}
aic = {}
bic = {}
loglik = {}
lbox = {}
noarch = {}

for pair in pairs:
     df = import_data(fx_pair=pair)
     df.dropna(inplace=True)
     returns = df['log_ret'] * 100
     returns[0] = 0.000
     gjr = arch_model(returns, mean='Zero', vol='GARCH', p=1,q=1,o=1, dist = 'Studentst')
     gjr_fit = gjr.fit()
     lbox_test = sm.stats.acorr_ljungbox(gjr_fit.resid, lags=[1,2,3], return_df=False)
     
     model[pair] = 'GARCH(1,1)'
     aic[pair] = np.round(gjr_fit.aic,3)
     bic[pair] = np.round(gjr_fit.bic,3)
     loglik[pair] = np.round(gjr_fit.loglikelihood,3)
     lbox[pair] = np.round(lbox_test[1][0],3)

gjrdf = pd.DataFrame([model,aic,bic,loglik,lbox], index=['Model','AIC', 'BIC', 'LogL','Ljung-Box (1)'])

Iteration:      1,   Func. Count:      7,   Neg. LLF: 11632.775452308942
Iteration:      2,   Func. Count:     14,   Neg. LLF: 8134.576326687024
Iteration:      3,   Func. Count:     21,   Neg. LLF: 5721.62182775618
Iteration:      4,   Func. Count:     28,   Neg. LLF: 3903.70925125576
Iteration:      5,   Func. Count:     34,   Neg. LLF: 8109.990251157093
Iteration:      6,   Func. Count:     42,   Neg. LLF: 55972.53071663463
Iteration:      7,   Func. Count:     51,   Neg. LLF: 4091.2370128412344
Iteration:      8,   Func. Count:     58,   Neg. LLF: 13275.357887406146
Iteration:      9,   Func. Count:     66,   Neg. LLF: 3854.6032514149783
Iteration:     10,   Func. Count:     73,   Neg. LLF: 3842.920330958757
Iteration:     11,   Func. Count:     79,   Neg. LLF: 3842.7627518551917
Iteration:     12,   Func. Count:     85,   Neg. LLF: 3842.7012709377796
Iteration:     13,   Func. Count:     91,   Neg. LLF: 3842.663746228166
Iteration:     14,   Func. Count:     97,   Neg. LLF: 3842.6

In [168]:
garchdf

,USDRUB,USDZAR,USDBRL,EURUSD,USDJPY,GBPUSD
Model,"GARCH(1,1)","GARCH(1,1)","GARCH(1,1)","GARCH(1,1)","GARCH(1,1)","GARCH(1,1)"
AIC,7719.554,16009.926,12298.198,9154.027,9146.87,8826.097
BIC,7745.138,16036.547,12324.104,9180.637,9173.491,8852.718
LogL,-3855.777,-8000.963,-6145.099,-4573.013,-4569.435,-4409.048
Ljung-Box (1),0.567,0.202,0.0,0.294,0.007,0.016


In [169]:
gjrdf

,USDRUB,USDZAR,USDBRL,EURUSD,USDJPY,GBPUSD
Model,"GARCH(1,1)","GARCH(1,1)","GARCH(1,1)","GARCH(1,1)","GARCH(1,1)","GARCH(1,1)"
AIC,7695.326,15996.511,12258.381,9153.266,9145.42,8821.444
BIC,7727.306,16029.787,12290.763,9186.529,9178.696,8854.72
LogL,-3842.663,-7993.255,-6124.19,-4571.633,-4567.71,-4405.722
Ljung-Box (1),0.567,0.202,0.0,0.294,0.007,0.016


In [191]:
df = import_data(fx_pair='USDJPY')
df.dropna(inplace=True)
returns = df['log_ret'] * 100
returns[0] = 0.000
gjr = arch_model(returns, mean='Zero', vol='GARCH', p=1,q=1,o=1, dist = 'Studentst')
gjr_fit = gjr.fit()
gjr_fit

Iteration:      1,   Func. Count:      7,   Neg. LLF: 8129.419557903359
Iteration:      2,   Func. Count:     15,   Neg. LLF: 9965.110318555142
Iteration:      3,   Func. Count:     22,   Neg. LLF: 7501.340376749749
Iteration:      4,   Func. Count:     30,   Neg. LLF: 4690.040731902739
Iteration:      5,   Func. Count:     37,   Neg. LLF: 4674.903825990782
Iteration:      6,   Func. Count:     44,   Neg. LLF: 4590.505415377436
Iteration:      7,   Func. Count:     51,   Neg. LLF: 9977.655511772307
Iteration:      8,   Func. Count:     58,   Neg. LLF: 4573.598732649718
Iteration:      9,   Func. Count:     65,   Neg. LLF: 4576.406058713219
Iteration:     10,   Func. Count:     73,   Neg. LLF: 4696.350837463877
Iteration:     11,   Func. Count:     81,   Neg. LLF: 4567.715488217296
Iteration:     12,   Func. Count:     87,   Neg. LLF: 4567.710631484144
Iteration:     13,   Func. Count:     93,   Neg. LLF: 4567.711779812718
Iteration:     14,   Func. Count:    100,   Neg. LLF: 4567.71014

                        Zero Mean - GJR-GARCH Model Results                         
Dep. Variable:                      log_ret   R-squared:                       0.000
Mean Model:                       Zero Mean   Adj. R-squared:                  0.000
Vol Model:                        GJR-GARCH   Log-Likelihood:               -4567.71
Distribution:      Standardized Student's t   AIC:                           9145.42
Method:                  Maximum Likelihood   BIC:                           9178.70
                                              No. Observations:                 5740
Date:                      Fri, Apr 01 2022   Df Residuals:                     5740
Time:                              13:53:55   Df Model:                            0
                               Volatility Model                              
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
o